# Парсинг сайта

### Бибилиотеки

- requests - для отправки HTTP-запросов к веб-сайтам

- BeautifulSoup - для парсинга и разбора HTML-кода

- pandas - для работы с данными в табличном формате (DataFrame)

- re - для работы с регулярными выражениями (поиск и замена текста по шаблонам)

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Отправляем get запрос на сайт
url = "https://naked-science.ru/article/"
page = requests.get(url)

# Получаем HTML-код страницы
text = page.text
soup = BeautifulSoup(text, "html.parser")

# Выкидываем ненужные теги
for script in soup(["script", "style", "meta", "link", "noscript", "header", "head"]):
    script.decompose()

# Выводим страницу в упрощенном виде
print(soup.prettify()[:100000])

<!DOCTYPE html>
,<html lang="ru">
, <body class="wp-singular page-template page-template-category-all-articles page-template-category-all-articles-php page page-id-154361 wp-embed-responsive wp-theme-shesht">
,  <!-- Shadow -->
,  <div class="shadow">
,  </div>
,  <!-- /Shadow -->
,  <div class="ads_header mob">
,   <!-- Yandex.RTB -->
,   <!-- Yandex.RTB R-A-265695-52 -->
,   <div id="yandex_rtb_R-A-265695-52">
,   </div>
,  </div>
,  <div class="ads_header pc">
,  </div>
,  <!-- Main Container -->
,  <div id="main-container">
,   <!-- Google Tag Manager (noscript) -->
,   <!-- End Google Tag Manager (noscript) -->
,   <!-- Main Content -->
,   <section id="main-content">
,    <!-- Header -->
,    <div class="header-wrapp">
,     <div class="page_load_line">
,     </div>
,    </div>
,    <!-- /Header -->
,    <!-- Нативный баннер, переместил сюда Руслан -->
,    <!-- Конец нативного баннера -->
,    <section id="content">
,     <div class="page-category">
,      <div class="box-contai

Смотрим вывод страницы, и ищем нужные теги

### Заголовок URL и рейтинг

Нашли его в теге div news-item-title -> h3 -> a, в атрибуте href ссылка на саму статью, и там же (news-item-title h3) в span лежит рейтинг

### Текст статьи

div news-item-excerpt -> p

### Дата публикации

div meta-item -> span echo_date

### Комментарии

div commnets-count во втром span без класса

In [3]:
articles = soup.find_all('div', class_='news-item')
print(f"Найдено статей: {len(articles)}")

Найдено статей: 20


## Проверка

In [4]:
def parse_articles_function(articles):
    for i, article in enumerate(articles[:3]):
        print(f"\n=== Статья {i+1} ===")
        
        # Заголовок
        title_elem = article.find('div', class_='news-item-title')
        if title_elem:
            title_link = title_elem.find('a')
            if title_link:
                # Копируем элемент чтобы не испортить оригинал
                title_clone = BeautifulSoup(str(title_link), 'html.parser').find('a')
                
                # Удаляем span с рейтингом из клона
                rating_span = title_clone.find('span', class_='index_importance_news')
                if rating_span:
                    rating_span.decompose()  # Удаляем рейтинг из заголовка
                
                # Получаем чистый заголовок
                clean_title = title_clone.get_text(strip=True)
                print(f"Заголовок: {clean_title}")
                
                # Ссылка
                url = title_link.get('href')
                print(f"URL: {url}")
        
        # Текст
        excerpt_elem = article.find('div', class_='news-item-excerpt')
        if excerpt_elem:
            description = excerpt_elem.get_text(strip=True)
            print(f"Текст: {description}...")  # Первые 100 символов
        
        # Дата публикации
        date_elem = article.find('span', class_='echo_date')
        if date_elem:
            published_at = date_elem.get_text(strip=True)
            print(f"Дата: {published_at}")
        
        # Рейтинг
        rating_elem = article.find('span', class_='index_importance_news')
        if rating_elem:
            rating = rating_elem.get_text(strip=True)
            print(f"Рейтинг: {rating}")
        
        # Комментарии
        comments_elem = article.find('div', class_='commnets-count')
        if comments_elem:
            # Ищем число комментариев - обычно второй span
            spans = comments_elem.find_all('span')
            if len(spans) > 1:
                comments_count = spans[1].get_text(strip=True)
                print(f"Комментарии: {comments_count}")
        else:
            print("Комментарии: 0")
        
        print("-" * 50)


parse_articles_function(articles)


,=== Статья 1 ===
,Заголовок: Ученые обнаружили «акценты» у львов из разных стран
,URL: https://naked-science.ru/article/biology/uchenye-obnaruzhili-aktse
,Текст: Исследователи из Великобритании и Танзании разработали метод автоматической классификации львиных вокализаций с помощью машинного обучения. Новый подход позволил не только уточнить структуру рева, выделив в нем ранее неизвестный элемент, но и повысить точность идентификации отдельных особей до 87%. Попутно выяснилось, что хищники из разных регионов Африки «разговаривают» по-разному....
,Дата: 24 ноября, 20:19
,Рейтинг: 3.5
,Комментарии: 0
,--------------------------------------------------
,
,=== Статья 2 ===
,Заголовок: Искусственный интеллект обретает здравомыслие: новый метод заставил нейросети сомневаться в своих ответах
,URL: https://naked-science.ru/article/column/iskusstvennyj-int-o
,Текст: Команда исследователей из МИСиС и МФТИ с коллегами разработала новый метод, который значительно повышает надежность нейронных сет

Окей, по проверке все хорошо, теперь можно создавать бд и переходить к парсингу нескольких страниц (т.к. на главной показывается только первые 20)

## Создаем БД

In [9]:
import sqlite3
import uuid
from datetime import datetime
import time
import random

conn = sqlite3.connect('news.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS articles (
        guid TEXT PRIMARY KEY,
        title TEXT NOT NULL,
        description TEXT,
        url TEXT UNIQUE NOT NULL,
        published_at TEXT,
        comments_count INTEGER DEFAULT 0,
        created_at_utc TEXT NOT NULL,
        rating REAL
    )
''')
conn.commit()

## Парсим данные

Теперь берем 250 страниц, чтобы получить 5000 статей

In [10]:
all_articles_data = []
page_num = 1
max_pages = 250  # ~20 статей на странице * 250 = 5000

while page_num <= max_pages and len(all_articles_data) < 5000:
    print(f"Парсим страницу {page_num}...")
    
    # Формируем URL страницы
    if page_num == 1:
        url = "https://naked-science.ru/article/"
    else:
        url = f"https://naked-science.ru/article/page/{page_num}/"
    
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.text, "html.parser")
        
        # Удаляем ненужные теги
        for script in soup(["script", "style", "meta", "link", "noscript", "header", "head"]):
            script.decompose()
        
        articles = soup.find_all('div', class_='news-item')
        print(f"Найдено статей на странице: {len(articles)}")
        
        if len(articles) == 0:
            print("Статьи закончились, останавливаемся")
            break
        
        # Парсим каждую статью на странице
        for article in articles:
            try:
                # Заголовок и URL
                title_elem = article.find('div', class_='news-item-title')
                if title_elem:
                    title_link = title_elem.find('a')
                    if title_link:
                        # Копируем и чистим заголовок от рейтинга
                        title_clone = BeautifulSoup(str(title_link), 'html.parser').find('a')
                        rating_span = title_clone.find('span', class_='index_importance_news')
                        if rating_span:
                            rating_span.decompose()
                        clean_title = title_clone.get_text(strip=True)
                        url = title_link.get('href')
                    else:
                        continue
                else:
                    continue
                
                # Текст статьи
                excerpt_elem = article.find('div', class_='news-item-excerpt')
                description = excerpt_elem.get_text(strip=True) if excerpt_elem else ""
                
                # Дата
                date_elem = article.find('span', class_='echo_date')
                published_at = date_elem.get_text(strip=True) if date_elem else ""
                
                # Рейтинг
                rating_elem = article.find('span', class_='index_importance_news')
                rating = None
                if rating_elem:
                    try:
                        rating = float(rating_elem.get_text(strip=True))
                    except:
                        rating = None
                
                # Комментарии
                comments_elem = article.find('div', class_='commnets-count')
                comments_count = 0
                if comments_elem:
                    spans = comments_elem.find_all('span')
                    if len(spans) > 1:
                        comments_text = spans[1].get_text(strip=True)
                        if 'тыс' in comments_text:
                            try:
                                count = float(comments_text.replace('тыс', '').replace(',', '.').strip()) * 1000
                                comments_count = int(count)
                            except:
                                comments_count = 0
                        else:
                            try:
                                comments_count = int(comments_text)
                            except:
                                comments_count = 0
                
                # Сохраняем данные
                article_data = (
                    str(uuid.uuid4()),
                    clean_title,
                    description,
                    url,
                    published_at,
                    comments_count,
                    datetime.utcnow().isoformat(),
                    rating
                )
                
                # Пытаемся добавить в базу
                try:
                    cursor.execute('''
                        INSERT INTO articles 
                        (guid, title, description, url, published_at, comments_count, created_at_utc, rating)
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                    ''', article_data)
                    all_articles_data.append(article_data)
                except sqlite3.IntegrityError:
                    # URL уже существует, пропускаем
                    pass
                    
            except Exception as e:
                print(f"Ошибка при парсинге статьи: {e}")
                continue
        
        conn.commit()
        print(f"Уже собрано статей: {len(all_articles_data)}")
        
        # Задержка чтобы не заблокировали
        time.sleep(random.uniform(1, 3))
        
        page_num += 1
        
    except Exception as e:
        print(f"Ошибка при загрузке страницы {page_num}: {e}")
        break

conn.close()

Парсим страницу 1...
,Найдено статей на странице: 20
,Уже собрано статей: 0
,Парсим страницу 2...
,Найдено статей на странице: 20
,Уже собрано статей: 20
,Парсим страницу 3...
,Найдено статей на странице: 20
,Уже собрано статей: 40
,Парсим страницу 4...
,Найдено статей на странице: 20
,Уже собрано статей: 60
,Парсим страницу 5...
,Ошибка при загрузке страницы 5: HTTPSConnectionPool(host='naked-science.ru', port=443): Max retries exceeded with url: /article/page/5/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000019519095750>, 'Connection to naked-science.ru timed out. (connect timeout=None)'))


Тут сайт нам вежливо напомнил о юзер агенте на 5 странице, исправляем

In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import uuid
from datetime import datetime
import time
import random

# Создаем базу данных
conn = sqlite3.connect('news.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS articles (
        guid TEXT PRIMARY KEY,
        title TEXT NOT NULL,
        description TEXT,
        url TEXT UNIQUE NOT NULL,
        published_at TEXT,
        comments_count INTEGER DEFAULT 0,
        created_at_utc TEXT NOT NULL,
        rating REAL
    )
''')
conn.commit()

# Список User-Agent для ротации
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15'
]

# Парсим много страниц
all_articles_count = 0
page_num = 1
max_pages = 300

while page_num <= max_pages and all_articles_count < 5000:
    print(f"Парсим страницу {page_num}...")
    
    # Формируем URL
    if page_num == 1:
        url = "https://naked-science.ru/article/"
    else:
        url = f"https://naked-science.ru/article/page/{page_num}/"
    
    try:
        # Случайный User-Agent и задержка
        headers = {
            'User-Agent': random.choice(user_agents),
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
            'Accept-Encoding': 'gzip, deflate',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1',
        }
        
        # Увеличиваем таймаут и добавляем повторные попытки
        session = requests.Session()
        session.headers.update(headers)
        
        response = session.get(url, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Удаляем ненужные теги
        for script in soup(["script", "style", "meta", "link", "noscript", "header", "head"]):
            script.decompose()
        
        articles = soup.find_all('div', class_='news-item')
        print(f"Найдено статей на странице: {len(articles)}")
        
        if len(articles) == 0:
            print("Статьи закончились, останавливаемся")
            break
        
        # Парсим каждую статью на странице
        page_articles_count = 0
        for article in articles:
            try:
                # Заголовок и URL
                title_elem = article.find('div', class_='news-item-title')
                if title_elem:
                    title_link = title_elem.find('a')
                    if title_link:
                        # Копируем и чистим заголовок от рейтинга
                        title_clone = BeautifulSoup(str(title_link), 'html.parser').find('a')
                        rating_span = title_clone.find('span', class_='index_importance_news')
                        if rating_span:
                            rating_span.decompose()
                        clean_title = title_clone.get_text(strip=True)
                        url = title_link.get('href')
                    else:
                        continue
                else:
                    continue
                
                # Текст статьи
                excerpt_elem = article.find('div', class_='news-item-excerpt')
                description = excerpt_elem.get_text(strip=True) if excerpt_elem else ""
                
                # Дата
                date_elem = article.find('span', class_='echo_date')
                published_at = date_elem.get_text(strip=True) if date_elem else ""
                
                # Рейтинг
                rating_elem = article.find('span', class_='index_importance_news')
                rating = None
                if rating_elem:
                    try:
                        rating = float(rating_elem.get_text(strip=True))
                    except:
                        rating = None
                
                # Комментарии
                comments_elem = article.find('div', class_='commnets-count')
                comments_count = 0
                if comments_elem:
                    spans = comments_elem.find_all('span')
                    if len(spans) > 1:
                        comments_text = spans[1].get_text(strip=True)
                        if 'тыс' in comments_text:
                            try:
                                count = float(comments_text.replace('тыс', '').replace(',', '.').strip()) * 1000
                                comments_count = int(count)
                            except:
                                comments_count = 0
                        else:
                            try:
                                comments_count = int(comments_text)
                            except:
                                comments_count = 0
                
                # Сохраняем данные
                try:
                    cursor.execute('''
                        INSERT INTO articles 
                        (guid, title, description, url, published_at, comments_count, created_at_utc, rating)
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                    ''', (
                        str(uuid.uuid4()),
                        clean_title,
                        description,
                        url,
                        published_at,
                        comments_count,
                        datetime.utcnow().isoformat(),
                        rating
                    ))
                    page_articles_count += 1
                    all_articles_count += 1
                except sqlite3.IntegrityError:
                    # URL уже существует, пропускаем
                    pass
                    
            except Exception as e:
                print(f"Ошибка при парсинге статьи: {e}")
                continue
        
        conn.commit()
        print(f"Добавлено статей с этой страницы: {page_articles_count}")
        print(f"Всего собрано статей: {all_articles_count}")
        
        # Случайная задержка от 3 до 8 секунд
        delay = random.uniform(3, 8)
        print(f"Ждем {delay:.1f} секунд...")
        time.sleep(delay)
        
        page_num += 1
        
    except requests.exceptions.RequestException as e:
        print(f"Ошибка сети на странице {page_num}: {e}")
        print("Ждем 30 секунд и пробуем снова...")
        time.sleep(30)
        continue
        
    except Exception as e:
        print(f"Ошибка при загрузке страницы {page_num}: {e}")
        break

conn.close()

print(f"\n=== РЕЗУЛЬТАТ ===")
print(f"Всего собрано статей: {all_articles_count}")

# Проверяем базу
conn = sqlite3.connect('news.db')
cursor = conn.cursor()
cursor.execute('SELECT COUNT(*) FROM articles')
total_in_db = cursor.fetchone()[0]
conn.close()

print(f"Записей в базе: {total_in_db}")

Парсим страницу 1...
,Ошибка сети на странице 1: HTTPSConnectionPool(host='naked-science.ru', port=443): Max retries exceeded with url: /article/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000025B02F5BF10>, 'Connection to naked-science.ru timed out. (connect timeout=10)'))
,Ждем 30 секунд и пробуем снова...
,Парсим страницу 1...
,Найдено статей на странице: 20
,Добавлено статей с этой страницы: 0
,Всего собрано статей: 0
,Ждем 4.5 секунд...
,Парсим страницу 2...
,Найдено статей на странице: 20
,Добавлено статей с этой страницы: 0
,Всего собрано статей: 0
,Ждем 4.8 секунд...
,Парсим страницу 3...
,Найдено статей на странице: 20
,Добавлено статей с этой страницы: 0
,Всего собрано статей: 0
,Ждем 5.9 секунд...
,Парсим страницу 4...
,Ошибка сети на странице 4: HTTPSConnectionPool(host='naked-science.ru', port=443): Max retries exceeded with url: /article/page/4/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000025B040AF

In [11]:
import pandas as pd

# Подключаемся к базе
conn = sqlite3.connect('news.db')

# Сохраняем в CSV файл
df = pd.read_sql('SELECT * FROM articles', conn)
df.to_csv('naked_science_articles.csv', index=False, encoding='utf-8-sig')

for i, row in df.head(3).iterrows():
    print(f"\nСТАТЬЯ {i+1}:")
    print(f"Заголовок: {row['title']}")
    print(f"URL: {row['url']}")
    print(f"Дата: {row['published_at']}")
    print(f"Рейтинг: {row['rating']}")
    print(f"Комментарии: {row['comments_count']}")
    print(f"Текст: {row['description'][:100]}")
    print(f"GUID: {row['guid']}")
    print(f"Создано в БД: {row['created_at_utc']}")


,СТАТЬЯ 1:
,Заголовок: Ученые обнаружили «акценты» у львов из разных стран
,URL: https://naked-science.ru/article/biology/uchenye-obnaruzhili-aktse
,Дата: 24 ноября, 20:19
,Рейтинг: 3.5
,Комментарии: 0
,Текст: Исследователи из Великобритании и Танзании разработали метод автоматической классификации львиных во
,GUID: 6581a034-e949-476d-9427-c326a133d3b4
,Создано в БД: 2025-11-26T08:01:03.138087
,
,СТАТЬЯ 2:
,Заголовок: Искусственный интеллект обретает здравомыслие: новый метод заставил нейросети сомневаться в своих ответах
,URL: https://naked-science.ru/article/column/iskusstvennyj-int-o
,Дата: 24 ноября, 17:58
,Рейтинг: 4.9
,Комментарии: 0
,Текст: Команда исследователей из МИСиС и МФТИ с коллегами разработала новый метод, который значительно повы
,GUID: d838c5f4-7072-4d5a-8fea-557b57eeb921
,Создано в БД: 2025-11-26T08:01:03.140084
,
,СТАТЬЯ 3:
,Заголовок: Неандертальцы убили и съели женщин и детей из чужого племени
,URL: https://naked-science.ru/article/archeology/neandertaltsy-ubili-

Вроде все отображается корректно, 5841 запись

Рейтинг имеет 139 null значений, в коментариях пустые ячейки преобразованы в 0

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
,RangeIndex: 5841 entries, 0 to 5840
,Data columns (total 8 columns):
, #   Column          Non-Null Count  Dtype  
,---  ------          --------------  -----  
, 0   guid            5841 non-null   object 
, 1   title           5841 non-null   object 
, 2   description     5841 non-null   object 
, 3   url             5841 non-null   object 
, 4   published_at    5841 non-null   object 
, 5   comments_count  5841 non-null   int64  
, 6   created_at_utc  5841 non-null   object 
, 7   rating          5702 non-null   float64
,dtypes: float64(1), int64(1), object(6)
,memory usage: 365.2+ KB


In [14]:
df.head()

,guid,title,description,url,published_at,comments_count,created_at_utc,rating
0,6581a034-e949-476d-9427-c326a133d3b4,Ученые обнаружили «акценты» у львов из разных ...,Исследователи из Великобритании и Танзании раз...,https://naked-science.ru/article/biology/uchen...,"24 ноября, 20:19",0,2025-11-26T08:01:03.138087,3.5
1,d838c5f4-7072-4d5a-8fea-557b57eeb921,Искусственный интеллект обретает здравомыслие:...,Команда исследователей из МИСиС и МФТИ с колле...,https://naked-science.ru/article/column/iskuss...,"24 ноября, 17:58",0,2025-11-26T08:01:03.140084,4.9
2,3156e749-5ab5-4b83-87f5-24baa453081a,Неандертальцы убили и съели женщин и детей из ...,Анализ неандертальских костей из бельгийской п...,https://naked-science.ru/article/archeology/ne...,"24 ноября, 14:46",0,2025-11-26T08:01:03.140084,3.9
3,fb9fa848-99ab-408c-aff2-efea41f62fbe,"Ученые выяснили, как избежать выгорания студентов",Исследователи Института образования НИУ ВШЭ по...,https://naked-science.ru/article/column/kak-iz...,"24 ноября, 10:59",0,2025-11-26T08:01:03.141080,4.8
4,e56549d4-b130-4e4f-9e46-8e0534a35ef2,"Астрофизики, возможно, нашли следы самых масси...","В далекой галактике, которую ученые видят тако...",https://naked-science.ru/article/astronomy/nas...,"24 ноября, 10:34",0,2025-11-26T08:01:03.142077,5.8
